In [3]:
## License: Apache 2.0. See LICENSE file in root directory.
## Copyright(c) 2015-2017 Intel Corporation. All Rights Reserved.

###############################################
##      Open CV and Numpy integration        ##
###############################################

# import pyrealsense2 as rs
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
from time import perf_counter
import os
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
from sklearn import preprocessing
# from statsmodels import stats
from scipy import stats
import csv

ModuleNotFoundError: No module named 'cv2'

In [2]:
pip install --upgrade pip

  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyrealsense2

ERROR: Could not find a version that satisfies the requirement pyrealsense2 (from versions: none)
ERROR: No matching distribution found for pyrealsense2
Note: you may need to restart the kernel to use updated packages.


In [3]:
'''データ保存パス指定場所'''

# num = 0

# dir_path1 = "data/20220707/data1/img/img1/color"+str(num)
# dir_path2 = "data/20220707/data1/img/img1/depth"+str(num)
# basename = "img"
# ext = "png"
# delay = 1
# os.makedirs(dir_path1, exist_ok=True)
# os.makedirs(dir_path2, exist_ok=True)
# base_path1 = os.path.join(dir_path1, basename)
# base_path2 = os.path.join(dir_path2, basename)
# n = 0

# f0 = open("data/20220707/data1/mean/mean"+str(num)+".csv", "w")
# writer0 = csv.writer(f0)

# f1 = open("data/20220707/data1/variance/variance"+str(num)+".csv", "w")
# writer1 = csv.writer(f1)

'データ保存パス指定場所'

In [1]:
conda install scikit-image

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                                                      |failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - argon2-cffi -> python[version='>=2.7,<2.8.0a0|>=3.5,<3.6.0a0']
  - argon2-cffi-bindings -> python[version='>=2.7,<2.8.0a0|>=3.6,<3.7.0a0|>=3.5,<3.6.0a0']
  - attrs -> python[version='>=2.7,<2.8.0a0|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.5,<3.6.0a0']
  - backcall -> python[version='>=2.7,<2.8.0a0|>=3.5,<3.6.0

In [2]:
conda install scikit-image

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                                                                 \failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - argon2-cffi -> python[version='>=2.7,<2.8.0a0|>=3.5,<3.6.0a0']
  - argon2-cffi-bindings -> python[version='>=2.7,<2.8.0a0|>=3.6,<3.7.0a0|>=3.5,<3.6.0a0']
  - asttokens -> python[version='>=2.7,<2.8.0a0|>=3.6,<3.7.0a0|>=3.8,<3.9.0a0|>=3.9,<3.10.0a0|>=3.7,<3.8.0a0|>=3.5,<3.6.0a0']
  - attrs -> 

In [3]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == "RGB Camera":
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)

config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

if device_product_line == "L500":
    config.enable_stream(rs.stream.color, 960, 540, rs.format.bgr8, 30)
else:
    config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)
start = time.time()
t_init = perf_counter()

img_size = 168
split_cell = 28
cell_length = int(img_size / split_cell)

# 正規化メソッドの作成
mm_2d = preprocessing.MinMaxScaler()

In [4]:
try:
    while True:
        t = perf_counter() - t_init
        # print(n)

        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(
            cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET
        )

        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape

        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            resized_color_image = cv2.resize(
                color_image,
                dsize=(depth_colormap_dim[1], depth_colormap_dim[0]),
                interpolation=cv2.INTER_AREA,
            )
            images = np.hstack((resized_color_image, depth_colormap))
        else:
            images = np.hstack((color_image, depth_colormap))

        depth_image = depth_image[:, 80:-80]
        color_image = color_image[:, 80:-80]

        resized_depth = cv2.resize(depth_image, dsize=(img_size, img_size))
        resized_color = cv2.resize(color_image, dsize=(img_size, img_size))
        resized_depth_colormap = cv2.resize(depth_colormap, dsize=(img_size, img_size))

        var_list = []
        mean_list = []

        for i in range(split_cell):
            for j in range(split_cell):
                surface = resized_depth[:, cell_length * i : cell_length * (i + 1)][
                    cell_length * j : cell_length * (j + 1)
                ]
                # print(surface)
                variance = round(np.var(surface), 2)  # 分散計算
                image_mean = round(np.mean(surface), 2)  # 平均計算
                # print(variance)
                var_list.append(variance)
                mean_list.append(image_mean)

                # cv2.line(
                #     resized_depth_colormap,
                #     (0, cell_length * i),
                #     (img_size, cell_length * i),
                #     color=(0, 0, 255),
                
                # )
                # cv2.line(
                #     resized_depth_colormap,
                #     (cell_length * i, 0),
                #     (cell_length * i, img_size),
                #     color=(0, 0, 255),
                # )

        ################################################################################

        # mean_arr = np.reshape(mean_list, (28, 28))
        # var_arr = np.reshape(var_list, (28, 28))

        # for i in range(1, 27):
        #     for j in range(1, 27):
        #         mean_m = np.median([
        #             mean_arr[j-1][i-1], mean_arr[j-1][i], mean_arr[j-1][i+1], 
        #             mean_arr[j+1][i-1], mean_arr[j+1][i], mean_arr[j+1][i+1], 
        #             mean_arr[j][i-1], mean_arr[j][i+1]
        #             ])
        #         # print(mean_v)
        #         mean_arr[j][i] = mean_m

        # for i in range(1, 27):
        #     for j in range(1, 27):
        #         mean_v = np.sum([
        #             var_arr[j-1][i-1], var_arr[j-1][i], var_arr[j-1][i+1], 
        #             var_arr[j+1][i-1], var_arr[j+1][i], var_arr[j+1][i+1], 
        #             var_arr[j][i-1], var_arr[j][i+1]
        #             ])/8
        #         # print(mean_m)
        #         var_arr[j][i] = mean_v
        
        

        ################################################################################

        # # csvファイルとして保存
        # writer0.writerow(mean_list)
        # writer1.writerow(var_list)

        # # 動画保存
        # cv2.imwrite("{}_{}.{}".format(base_path1, n, ext), resized_color)
        # cv2.imwrite("{}_{}.{}".format(base_path2, n, ext), resized_depth_colormap)
        # n += 1

        ################################################################################

        # 取得一次元配列データを正規化
        # 画像サイズを28×28にリサイズ
        # mean_array = np.round(preprocessing.minmax_scale(mean_list), 3)
        # var_array = np.round(preprocessing.minmax_scale(var_list), 3)
        # var_arr = np.reshape(var_array, (28, 28))
        # plane_array = var_arr[:, 10:18][10:18]
        # d = np.round(np.var(plane_array), 3)
        # plane_p = np.reshape(plane_array, (1, 64))
        # print(plane_p)
        # print(d)

        ##################################################################################


        # #推定連長と信念の計算
        # maxes, current_belief = bcp.update(d)
        # result = np.diff(maxes)
        # beliefs.append(current_belief)
        # print(result)

        # print(result)

        # print(color_image.shape)

        # # 画像中心の距離情報の取得
        # depth_data = np.round(resized_depth[img_size / 2, img_size / 2])

        # print(depth_data)

        # Show images
        cv2.namedWindow("RealSense", cv2.WINDOW_AUTOSIZE)
        cv2.imshow("RealSense", images)
        cv2.imshow("depth_color", resized_depth_colormap)
        cv2.imshow("color_image", resized_color)
        # cv2.imshow("transform", var_arr)
        cv2.waitKey(1)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            pipeline.stop()
            cv2.destroyAllWindows()
            break
#         time.sleep(0.01)


finally:
    # Stop streaming
    pipeline.stop()


RuntimeError: stop() cannot be called before start()